## Vegan Store  

In this code, we define a class called "VeganStore," a management system for handling a vegan store.

Within the __init__ method, we initialize lists for the products in stock and the recorded sales, and we load data from a JSON file. Additionally, upon startup, we display the list of all commands using "show_help()".

We then proceed with saving and loading data to/from the "data_store.json" JSON file.

The subsequent methods allow, in order:

Adding products to the stock (add_product);
Displaying the list of products (product_list);
Registering sales (register_sale);
Calculating profits (calculate_profits);
Showing the list of available commands, which is also displayed at the start of the program (show_help).


In [ ]:
import json

class VeganStore:
    """A class representing a vegan store with products and sales."""
    def __init__(self):
        """Initialize the vegan store with an empty list of products and sales. Load data from a file if available."""
        self.store = []
        self.sales = []
        self.load_data()
        self.show_help()
        
    def save_data(self):
        """Save the current state of the store and sales to a JSON file."""
        try:
            with open("data_store.json", "w") as file:
                json.dump({"negozio": self.store, "vendite": self.sales}, file)
        except FileNotFoundError:
            print("File non trovato.")
        except Exception as e:
            print(f"Errore durante il salvataggio dei dati: {e}")
    
    def load_data(self):
        """Load data from a JSON file into the store and sales lists."""
        try:
            with open("data_store.json", "r") as file:
                data = json.load(file)
                self.store = data.get("negozio", [])
                self.sales = data.get("vendite", [])
        except FileNotFoundError:
            pass
        except Exception as e:
            print(f"Errore durante il caricamento dei dati: {e}")
    
    def add_product(self, name, quantity, purchase_price, sale_price):
        """Add a product to the store or update the quantity if the product already exists."""
        while True:
            try:
                quantity = int(quantity)
                purchase_price = float(purchase_price)
                if purchase_price < 0:
                    raise ValueError("Errore: Il prezzo di acquisto non può essere negativo.")
                sale_price = float(sale_price)
                if sale_price < 0:
                    raise ValueError("Errore: Il prezzo di vendita non può essere negativo.")
                break
            except ValueError as e:
                print(f"Errore: {e}")
                quantity = input("Inserisci la quantità corretta: ")
                purchase_price = input("Inserisci il prezzo di acquisto corretto: ")
                sale_price = input("Inserisci il prezzo di vendita corretto: ")

        if quantity < 1:
            print("Errore: Assicurati che la quantità sia positiva.")
            return

        for product in self.store:
            if product["nome"] == name:
                product["quantità"] += quantity
                break
        else:
            new_product = {"nome": name, "quantità": quantity, "prezzo di acquisto": purchase_price, "prezzo di vendita": sale_price}
            self.store.append(new_product)
        print(f"Aggiunto: {name} x {quantity}")
        self.save_data()
    
    def products_list(self):
        """Display a formatted list of products in the store."""
        print("PRODOTTO\tQUANTITA'\tPREZZO")
        for product in self.store:
            print(f"{product['nome']:<10}\t{product['quantità']:<8}\t${product['prezzo di vendita']:.2f}")

    
    def register_sale(self, sales):
        """Register a sale by updating the store quantities and recording the sale details."""
        total_sales = 0
        for sale in sales:
            name_product = sale["nome"]
            quantity_sold = sale["quantità"]

            product_found = False  # Aggiunto flag per tracciare se il prodotto è stato trovato
                
            for product in self.store:
                if product["nome"] == name_product:
                    if product["quantità"] >= quantity_sold:
                        product["quantità"] -= quantity_sold
                        total_sales += quantity_sold * product["prezzo di vendita"]
                        product_found = True
                        break
                    else: 
                        print(f"Errore: quantità insufficiente di {name_product} in magazzino.")
                        return
            if not product_found:  # Se il prodotto non è stato trovato, stampa l'errore
                print(f"Errore: {name_product} non presente in magazzino.")
                return

        print("Vendita registrata.")
        for sale in sales:
            print(f"{sale['quantità']} x {sale['nome']}: €{sale['quantità'] * product['prezzo di vendita']:.2f}")
        print(f"total: €{total_sales:.2f}")

        self.sales.append({"vendita": sales, "totale": total_sales})
        self.save_data()
            
    def calculate_profits(self):
        """Calculate and display the gross and net profits of the store."""
        total_sales = sum(sale["totale"] for sale in self.sales)
        
        total_purchase_cost = sum(
            sum(item["quantità"] * (item.get("prezzo di acquisto", 0) or product.get("prezzo di acquisto", 0))
                for item in sale["vendita"] if item["quantità"] > 0  # Considera solo vendite con quantità positive
                for product in self.store if product["nome"] == item["nome"])
            for sale in self.sales)
                
        gross_profit = total_sales
        net_profit = total_sales - total_purchase_cost
        
        print(f"Profitto: lordo = €{gross_profit:.2f} netto = €{net_profit:.2f}")

        
    def show_help(self):
        """Display a list of available commands and their descriptions."""
        print("Sono disponibili i seguenti comandi:")
        print("aggiungi: aggiungere un prodotto al magazzino")
        print("lista: elenca i prodotti in magazzino")
        print("vendita: registra una vendita effettuata")
        print("profitti: mostra i profitti totali")
        print("aiuto: mostra i comandi possibili")
        print("chiudi: esci dal programma")



This second piece of code is a script that creates an instance of the “VeganStore” class and starts a while loop to interact with the user through console commands.  

A “store” object of the “VeganStore” class is waxed to manage the store's operations. The program enters a while loop that continues until the user enters the “close” command.

The available commands are as follows:  
- *"vendita”*: To record a sale, requires the product name and quantity sold.
- *"profitti”*: To calculate and display the profits of the store.
- *"aggiungi”*: To add a new product to the store, requires name, quantity, purchase price and sale price.
- *"lista”*: To display the formatted list of products in the warehouse.
- *"aiuto”*: To display the list of available commands and their descriptions.
- *“Chiudi‘*: The program exits the loop and prints ’Buona giornata!”.
- Other invalid commands generate an error message.

In [ ]:
cmd = ""
if __name__ == "__main__":
    store = VeganStore()
    
    while cmd != "chiudi":
        cmd = input("Inserisci un comando: ").strip().lower()
        
        if cmd == "vendita":
            sales = []
            while True:
                product_name = input("Nome del prodotto: ")
                quantity_sold = int(input("Quantità: "))
                
                sales.append({"nome": product_name, "quantità": quantity_sold})
                
                other_product = input("Aggiungere un nuovo prodotto? (si/no): ").strip().lower()
                if other_product != "si":
                    break
                
            store.register_sale(sales)
        
        elif cmd == "profitti":
            store.calculate_profits()
            
        elif cmd == "aggiungi":
            product_name = input("Nome del prodotto: ")
            
            while True:
                try:
                    quantity = int(input("Quantità: "))
                    break
                except ValueError:
                    print("Inserire un numero intero.")
            purchase_price = float(input("Prezzo di acquisto: "))
            sale_price = float(input("Prezzo di vendita: "))
            
            store.add_product(product_name, quantity, purchase_price, sale_price)
        
        elif cmd == "lista":
            store.products_list()
        
        elif cmd == "aiuto":
            store.show_help()
        
        elif cmd != "chiudi":
            print("Comando non valido. Digitare aiuto per l'elenco dei comandi disponibili.")
            
    print("Buona giornata!")